# Assignment 4 
### Team Memebers:
 - Bhargav Patel - CS23MTECH11026
 - Manan Patel - CS23MTECH14006
 - Hrishikesh Hemke - CS23MTECH14003
 - Yug Patel - CS23MTECH14019

In [11]:
import numpy as np
from sympy import Matrix
import sys
import pandas as pd

In [2]:
import numpy as np
from sympy import Matrix
import sys
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

class Simplex_Algo():

    # class variables
    A = None
    b = None
    c = None
    Tolerance = 1e-6  # Small constant for numerical stability

    def __init__(self, filePath):
        # constructor to initialize the simplex algorithm with a given file path

        # Read cSV file into a NumPy array
        nArr = pd.read_csv(filePath, header=None).to_numpy()

        self.c = nArr[0:1, :-1].ravel()
        self.b = nArr[1:, -1:].T.ravel()
        self.A = nArr[1:, :-1]

    def retrieve_a_feasible_point(self, b):

        if np.all((b >= 0)):
            return np.zeros(self.c.shape)  # Return zero vector as a feasible point
        else:
            for _ in range(50):
                # Randomly select rows from A and corresponding elements from b
                random_indices = np.random.choice(self.A.shape[0], self.A.shape[1])
                random_A = self.A[random_indices]
                random_b = b[random_indices]

                try:
                    # Attempt to find a feasible point using a random subset of constraints
                    possible_X = np.dot(np.linalg.inv(random_A), random_b)

                    # check if the point satisfies all constraints
                    if (np.all((np.dot(self.A, possible_X) - b <= 0))):
                        return possible_X
                    else:
                        continue
                except:
                    pass

    def has_degeneracy(self, b):
        # Method to check if the problem has degeneracy

        # Find indices where the constraints are satisfied with equality
        equality_indices = np.where(np.abs(np.dot(self.A, (self.retrieve_a_feasible_point(b))) - b) < self.Tolerance)[0]

        # check if the number of equality indices is equal to the number of variables
        return len(equality_indices) != self.A.shape[1]

    def correct_degeneracy(self, b):
        # Method to correct degeneracy in the linear programming problem

        # calculate the number of rows to be modified
        rows_modified = (self.A.shape[0]) - (self.A.shape[1])

        n = 0

        while True:
            n += 1
            temp_b = b.astype(np.float64)

            if n < 1000:
                temp_b[:rows_modified] += np.random.uniform(self.Tolerance, self.Tolerance * 10, size=rows_modified)
            else:
                temp_b[:rows_modified] += np.random.uniform(0.1, 10, size=rows_modified)

            if not self.has_degeneracy(temp_b):
                print('Degeneracy removed')
                return temp_b

    def find_direction(self, b, X):
        # Method to find the direction for the next iteration of the simplex algorithm

        # Find indices where the constraints are satisfied with equality
        equality_indices = np.where(np.abs(np.dot(self.A, X) - b) < self.Tolerance)[0]

        # Extract the subset of constraints for the current equality indices
        A_bar = self.A[equality_indices]

        # calculate the direction matrix
        Z = -np.linalg.inv(np.transpose(A_bar))
        return Z

    def vertex_marching(self, b, X):
        # Method to perform vertex marching to find the optimal solution

        # Find the direction matrix for the current iteration
        Z = self.find_direction(b, X)

        # calculate costs for each direction
        costs = np.dot(Z, self.c)

        # Find indices of positive cost directions
        positive_cost_directions = np.where(costs > 0)[0]

        if not positive_cost_directions.any():
            return None  # Optimal vertex reached
    
        # choose the first positive cost direction
        v = Z[positive_cost_directions[0]]

        # check for unbounded polytope
        if not np.any(np.dot(self.A, v) > 0):
            print('Unbounded Polytope!!')
            sys.exit()

        # Extract non-equality indices and corresponding coefficients
        equality_indices = np.where(np.abs(np.dot(self.A, X) - b) < self.Tolerance)[0]
        not_equality_indices = ~np.isin(np.arange(len(self.A)), equality_indices)
        not_equal_A = self.A[not_equality_indices]
        not_equal_b = b[not_equality_indices]

        n = not_equal_b - np.dot(not_equal_A, X)
        d = np.dot(not_equal_A, v)
        n = n[np.where(d > 0)[0]]
        d = d[np.where(d > 0)[0]]
        s = n / d
        t = np.min(s[s >= 0])


        return X + t * v

    def fit(self):
        # Method to perform the simplex algorithm and find the optimal solution

        # correct degeneracy in the linear programming problem
        b = self.correct_degeneracy(self.b)

        # Retrieve a feasible point
        X = self.retrieve_a_feasible_point(b)
        print('\nInitial Feasible Point: ', tuple(X))

        print("\nJourney to Optimal Vertex (if it exists..)")
        while True:
            # Perform vertex marching to find the optimal solution
            V = self.vertex_marching(b, X)

            if V is None:
                break  # Optimal vertex reached
            else:
                # Print the current vertex and its cost
                print(f'z: {tuple(np.round(V, decimals=2))}; cost: {np.round(np.dot(V, self.c), decimals=2)}')
                X = V
        return X

# Example usage
algo = Simplex_Algo("Assignment4.csv")
X = algo.fit()
optimal_solution = list(map(float, (np.round(X, decimals=2)).flatten()))
print('\nThe Optimal Point is:', tuple(optimal_solution))
print('\nThe Optimal Value is: ', np.round(np.dot(algo.c, X), decimals=4))


Degeneracy removed

Initial Feasible Point:  (0.0, 0.0)

Journey to Optimal Vertex (if it exists..)
z: (4.0, 0.0); cost: 12.0
z: (0.0, 2.0); cost: 18.0

The Optimal Point is: (0.0, 2.0)

The Optimal Value is:  18.0


In [3]:
# Test input for above Output

# 3,9,0
# 1,4,8
# 1,2,4
# -1,0,0
# 0,-1,0